In [ ]:
# !pip uninstall -y torch torchvision torchaudio
# !pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0
# !MAX_JOBS=4 pip install flash-attn --no-build-isolation
# !pip install -U bitsandbytes transformers peft accelerate trl datasets sentencepiece wandb

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
!pip install transformers openai evaluate bert_score datasets rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=11cf5ffc6c513434926e83b1a702a97e387c0c4c8129d4a3a0835fce04c72ebe
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


## Load finetuned Aya

In [ ]:
adapter_repo_id = "linndfors/uk-sent-gender-swapper_aya-101"

config = PeftConfig.from_pretrained(adapter_repo_id)
base_model_id = config.base_model_name_or_path
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

base_model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

base_model.resize_token_embeddings(len(tokenizer))
inference_model = PeftModel.from_pretrained(base_model, adapter_repo_id)
inference_model = inference_model.to(device)

adapter_config.json:   0%|          | 0.00/767 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

...

model-00011-of-00011.safetensors:   0%|          | 0.00/4.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/6.30G [00:00<?, ?B/s]

### Testing

In [ ]:
input_sentence_2 = "Наша садівниця сьогодні була зовсім не в настрої"
_ = test_model_translation(input_sentence_2, inference_model, tokenizer, device)


Input prompt: 'Перефразуй це речення, змінивши гендерні сутності на протилежні (чоловічий <-> жіночий): Наша садівниця сьогодні була зовсім не в настрої'
Generating...
Generated Output: 'Наш садівник сьогодні був зовсім не в настрої.'
Generation time: 1.11 seconds


## Aya-101 (orig)

In [ ]:
# !pip uninstall -y torch torchvision torchaudio
# !pip install --upgrade --no-cache-dir --no-deps unsloth
# !pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0
# !MAX_JOBS=4 pip install flash-attn --no-build-isolation
!pip install -U bitsandbytes transformers peft accelerate trl datasets sentencepiece wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments


In [ ]:
MODEL_NAME = "CohereForAI/aya-101"

QUANTIZE_4BIT = True
USE_GRAD_CHECKPOINTS = True
TRAIN_BATCH_SIZE = 8
TRAIN_MAX_SEQ_LENGTH = 512
USE_FLASH_ATTENTION = True
GRAD_CC_STEPS = 2

In [ ]:
# quantization_config = None
# if QUANTIZE_4BIT:
#   quantization_config = BitsAndBytesConfig(
#       load_in_4bit=True,
#       bnb_4bit_quant_type="nf4",
#       bnb_4bit_use_double_quant=True,
#       bnb_4bit_compute_dtype=torch.bfloat16
#   )

In [ ]:
attn_implentation = None
if USE_FLASH_ATTENTION:
  attn_implementation="flash_attention_2"

In [ ]:
original_aya_model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

original_aya_model = original_aya_model.to(device)

original_aya_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

model-00004-of-00011.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00003-of-00011.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

model-00011-of-00011.safetensors:   0%|          | 0.00/4.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

# Evaluate

In [ ]:
from datasets import load_dataset
gender_df = load_dataset("csv", data_files="/content/test_parallel_for_llm_df.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
test_dataset = gender_df['train']

In [ ]:
test_dataset[0]

{'inputs': 'Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): офіцер',
 'targets': 'офіцерка'}

In [ ]:
import evaluate
from tqdm import tqdm
import bert_score


exact_match = evaluate.load("exact_match")
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def extract_job_entities(text):
    doc = ner_nlp(text)
    return set(ent.text.strip() for ent in doc.ents if ent.label_ == "JOB")


def compute_metrics(preds, refs, tokenizer):
    em = exact_match.compute(predictions=preds, references=refs)["exact_match"]
    bleu_score = bleu.compute(predictions=preds, references=refs)["bleu"]
    rouge_score = rouge.compute(predictions=preds, references=refs)
    bertscore = bert_score.score(preds, refs, lang="en", verbose=True)[2].mean().item()

    return {
        "Exact Match": round(em, 4),
        "BLEU": round(bleu_score, 4),
        "ROUGE-L": round(rouge_score["rougeL"], 4),
        "BERTScore (F1)": round(bertscore, 4),
    }


In [ ]:
def edit_ents(raw_output, orig_row):
    if not orig_row.strip().endswith(".") and raw_output.endswith("."):
        raw_output = raw_output[:-1]

    if orig_row[0].islower():
        return raw_output[0].lower() + raw_output[1:]
    return raw_output


def generate_swapped_response(input_text, inference_model, tokenizer, device=device):
  inputs = tokenizer(input_text, return_tensors="pt").to(device)

  with torch.no_grad():
      outputs = inference_model.generate(**inputs, max_new_tokens=130)

  generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return generated

In [ ]:
results = {
    # "finetuned": [],
    "references": [],
    "inputs": [],
    # "double_finetuned": [],
    "original": [],
    "double_original": []
}

for item in tqdm(test_dataset): #.select(range(5))
    prompt = item["inputs"]
    ref = item["targets"]

    if "Перефразуй це речення, змінивши гендерні сутності на протилежні (чоловічий <-> жіночий): " in prompt:
      input_str = prompt.split("Перефразуй це речення, змінивши гендерні сутності на протилежні (чоловічий <-> жіночий): ")
      prompt_base = "Перефразуй це речення, змінивши гендерні сутності на протилежні (чоловічий <-> жіночий): "
    elif "Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): " in prompt:
      input_str = prompt.split("Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): ")
      prompt_base = "Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): "
    else:
      print("ERROR")
      raise("No such line")

    results["references"].append(ref)
    results["inputs"].append(input_str[1])

    swapped_value = generate_swapped_response(prompt, original_aya_model, original_aya_tokenizer)
    swapped_value = edit_ents(swapped_value, input_str[1])

    results["original"].append(swapped_value)

    # swapped_value = generate_swapped_response(prompt, inference_model, tokenizer)
    # results["finetuned"].append(swapped_value)

    new_input = f"{prompt_base}{swapped_value}"

    double_swapped_value = generate_swapped_response(new_input, original_aya_model, original_aya_tokenizer)
    double_swapped_value = edit_ents(double_swapped_value, swapped_value)
    results["double_original"].append(double_swapped_value)

    # double_swapped_value = generate_swapped_response(new_input, inference_model, tokenizer)
    # results["double_finetuned"].append(double_swapped_value)

    # print(input_str[1], swapped_value, double_swapped_value)

100%|██████████| 523/523 [25:35<00:00,  2.94s/it]


In [ ]:
import pandas as pd

df = pd.DataFrame(results)

df.to_csv("MAIN_original_aya_for_the_test_parallel_dataset.csv", index=False, encoding="utf-8")

### Compute metrics

In [ ]:
import json

def generate_eval_res_stat(model_name, json_filename, tokenizer, target_col_name, results):

  all_metrics = {}

  print("\n📈 Evaluation Results:\n")

  print(f"🔹 {model_name.capitalize()} Model:")

  metrics = compute_metrics(results[model_name], results[target_col_name], tokenizer)
  all_metrics[model_name] = metrics

  for k, v in metrics.items():
      print(f"   {k}: {v}")
  print()

  with open(json_filename, "w", encoding="utf-8") as f:
      json.dump(all_metrics, f, indent=4, ensure_ascii=False)

In [ ]:
generate_eval_res_stat("finetuned", "MAIN_finetuned_evaluation_results.json", tokenizer, "references", results)


📈 Evaluation Results:

🔹 Finetuned Model:


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/13 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/9 [00:00<?, ?it/s]

done in 5.75 seconds, 90.91 sentences/sec
   Exact Match: 0.4398
   BLEU: 0.8211
   ROUGE-L: 0.2143
   BERTScore (F1): 0.9888
   Token Length Match Rate: 0.5641



In [ ]:
generate_eval_res_stat("double_finetuned", "MAIN_finetuned_aya_double_check_evaluation_results.json", tokenizer, "inputs", results)


📈 Evaluation Results:

🔹 Double_finetuned Model:


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/13 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/9 [00:00<?, ?it/s]

done in 4.84 seconds, 108.17 sentences/sec
   Exact Match: 0.5201
   BLEU: 0.8682
   ROUGE-L: 0.2143
   BERTScore (F1): 0.9913
   Token Length Match Rate: 0.6405



In [ ]:
generate_eval_res_stat("original", "MAIN_original_aya_evaluation_results.json", original_aya_tokenizer, "references", results)


📈 Evaluation Results:

🔹 Original Model:


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/9 [00:00<?, ?it/s]

done in 6.08 seconds, 86.08 sentences/sec
   Exact Match: 0.1491
   BLEU: 0.6472
   ROUGE-L: 0.2127
   BERTScore (F1): 0.968
   Token Length Match Rate: 0.3308



In [ ]:
generate_eval_res_stat("original", "MAIN_original_aya_evaluation_results.json", original_aya_tokenizer, "references", results)


📈 Evaluation Results:

🔹 Original Model:


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/15 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/9 [00:00<?, ?it/s]

done in 6.14 seconds, 85.16 sentences/sec
   Exact Match: 0.1931
   BLEU: 0.6644
   ROUGE-L: 0.2144
   BERTScore (F1): 0.9705



In [ ]:
generate_eval_res_stat("double_original", "MAIN_original_aya_double_check_evaluation_results.json", original_aya_tokenizer, "inputs", results)


📈 Evaluation Results:

🔹 Double_original Model:


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/15 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/9 [00:00<?, ?it/s]

done in 5.65 seconds, 92.64 sentences/sec
   Exact Match: 0.2658
   BLEU: 0.8068
   ROUGE-L: 0.21
   BERTScore (F1): 0.9693

